# 입문자를 위한, 파이썬/R 데이터 분석

## Part(2) - Day13  : 

]

## Today's mission

- 경진대회 실습 프로젝트 소개
- 실습 : 구내식당 식수 인원 예측 AI 경진대회
- GridSearchCV (하이퍼 파라미터 학습과 최적화)

]

## 구내식당 식수 인원 예측 AI 경진대회
정형 | 한국토지주택공사 | 식수예측 | MAE

https://dacon.io/competitions/official/235743/overview/description

In [ ]:
# 1) Import package
# 2) Data loading
# 3) Preprocessing
# 4) Modeling
# 5) Evaluation
# 6) Report / Submission

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv("data_dacon/train.csv", parse_dates=['일자'])
test = pd.read_csv("data_dacon/test.csv", parse_dates=['일자'])
submission = pd.read_csv("data_dacon/sample_submission.csv")

In [ ]:
print(train.shape)
train.head()

In [ ]:
print(test.shape)
test.head()

In [ ]:
print(submission.shape)
submission.head()

In [ ]:
1 + train['본사시간외근무명령서승인건수'] / train['본사정원수'] 

In [ ]:
train['hcount'] = train['본사정원수'] - train['본사휴가자수'] - train['본사출장자수'] - train['현본사소속재택근무자수']
train['hcount'] = train['hcount'] * (1 + train['본사시간외근무명령서승인건수'] / train['본사정원수'])

In [ ]:
train.head(3)

In [ ]:
train.isnull().sum()

In [ ]:
train.info()

In [ ]:
train['year'] = train['일자'].dt.year
train['month'] = train['일자'].dt.month
train['day'] = train['일자'].dt.day
train['dayofweek'] = train['일자'].dt.dayofweek

train.head(1)

In [ ]:
test['year'] = test['일자'].dt.year
test['month'] = test['일자'].dt.month
test['day'] = test['일자'].dt.day
test['dayofweek'] = test['일자'].dt.dayofweek

test['hcount'] = test['본사정원수'] - test['본사휴가자수'] - test['본사출장자수'] - test['현본사소속재택근무자수']
test['hcount'] = test['hcount'] * (1 + test['본사시간외근무명령서승인건수'] / test['본사정원수'])
test.head(1)

In [ ]:
plt.rc('font',family="Malgun Gothic")

In [ ]:
fig=plt.figure(figsize=(15,5))
sns.barplot(data=train, x='hcount', y='중식계')

In [ ]:
fig=plt.figure(figsize=(15,5))
sns.barplot(data=train, x='hcount', y='석식계')

In [ ]:
figure, ( (a1, a2, a3, a4), (a5, a6, a7, a8) )= plt.subplots(nrows=2 , ncols=4)

figure.set_size_inches(18, 10)

sns.barplot(data=train, x='year', y='중식계', ax=a1)
sns.barplot(data=train, x='month', y='중식계', ax=a2)
sns.barplot(data=train, x='day', y='중식계', ax=a3)
sns.barplot(data=train, x='dayofweek', y='중식계', ax=a4)

sns.barplot(data=train, x='year', y='석식계', ax=a5)
sns.barplot(data=train, x='month', y='석식계', ax=a6)
sns.barplot(data=train, x='day', y='석식계', ax=a7)
sns.barplot(data=train, x='dayofweek', y='석식계', ax=a8)

In [ ]:
train.corr()

In [ ]:
train.head(2)

In [ ]:
train.columns

In [ ]:
feature_cols_1 = ['본사휴가자수','본사출장자수','본사시간외근무명령서승인건수','dayofweek', 'hcount']
feature_cols_2 = ['본사휴가자수','본사시간외근무명령서승인건수','dayofweek', 'hcount']

In [ ]:
X_train_1 = train[feature_cols_1]
y_train_1 = train['중식계']

In [ ]:
X_train_2 = train[feature_cols_2]
y_train_2 = train['석식계']

In [ ]:
X_test_1 = test[feature_cols_1]
X_test_2 = test[feature_cols_2]

In [ ]:
# 모델링
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [ ]:
rf_reg_1 = RandomForestRegressor(n_jobs=-1, n_estimators=500, criterion='absolute_error')

In [ ]:
rf_reg_1

In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))
# Deprecated since version 1.0: Criterion “mae” was deprecated in v1.0 and will be removed in version 1.2. 
# Use criterion="absolute_error" which is equivalent.

In [ ]:
list_max_features = [0.1, 0.5, 0.9]
list_max_depth = [1, 3, 5]
n_estimators = 10# 300

list_hparam = []

for max_depth in list_max_depth:
    for max_features in list_max_features:
        rf = RandomForestRegressor(n_jobs=-1,criterion='squared_error', 
                                   n_estimators = n_estimators, 
                                   max_depth = max_depth, 
                                   max_features = max_features)
        
        score = cross_val_score(rf, X_train_1, y_train_1, cv=5).mean()
        
        result = {'score': score, 'n_esti':n_estimators, 'max_depth':max_depth,
                 'max_feat':max_features}
        
        list_hparam.append(result)
        print(max_depth, max_features, score, n_estimators, max_depth, max_features)

### GridSearchCV (하이퍼 파라미터 학습과 최적화)

- 사이킷런에서는 분류 알고리즘이나 회귀 알고리즘에 사용되는 하이퍼파라미터를 순차적으로 입력해 학습을 하고 측정을 하면서 가장 좋은 파라미터를 알려준다. 
- GridSearchCV가 없다면 max_depth 가 3일때 가장 최적의 스코어를 뽑아내는지 1일때 가장 최적인 스코어를  뽑아내는지 일일이 학습을 해야 한다. 
- 하지만 grid 파라미터 안에서 집합을 만들고 적용하면 최적화된 파라미터를 뽑아낼 수 있다.

참고 : https://rudolf-2434.tistory.com/10

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators' : [100,200,300],
    'max_depth' : [10,20,30,40,50,60,70,80,90],
    'max_features' : [0.1, 0.5, 0.9]    
}

rf_1 = RandomForestRegressor(n_jobs=-1, criterion='squared_error')

grid_cv = GridSearchCV(rf_1, param_grid = params, cv=5, n_jobs=-1)

grid_cv.fit(X_train_1, y_train_1)

In [ ]:
print(grid_cv.best_params_)
print(grid_cv.best_score_)

In [ ]:
print(score, n_estimators, max_depth, max_features)

In [ ]:
pd.DataFrame(list_hparam).sort_values(by='score')

In [ ]:
n_estimators = 300
num_epoch = 10
hyper_list_1 = []

for epoch in range(num_epoch):
    max_depth = np.random.randint(low=2, high=100)
    max_features = np.random.uniform(low=0.1, high=1.0)
    
    rf = RandomForestRegressor(n_jobs=-1, criterion='squared_error', 
                               n_estimators = n_estimators, 
                               max_depth = max_depth, 
                               max_features = max_features)
    
    score = cross_val_score(rf, X_train_1, y_train_1, cv=5).mean()
    
    h_params = {
        'epoch' : epoch,
        'score' : score,
        'n_estimators' : n_estimators,
        'max_depth' : max_depth,
        'max_features' : max_features
    }  
    
    hyper_list_1.append(h_params)

In [ ]:
pd.DataFrame.from_dict(hyper_list_1).sort_values(by='score')

In [ ]:
n_estimators = 300
num_epoch = 10
hyper_list_2 = []

for epoch in range(num_epoch):
    max_depth = np.random.randint(low=2, high=100)
    max_features = np.random.uniform(low=0.1, high=1.0)
    
    rf = RandomForestRegressor(n_jobs=-1, criterion='squared_error', 
                               n_estimators = n_estimators, 
                               max_depth = max_depth, 
                               max_features = max_features)
    
    score = cross_val_score(rf, X_train_2, y_train_2, cv=5).mean()
    
    h_params = {
        'epoch' : epoch,
        'score' : score,
        'n_estimators' : n_estimators,
        'max_depth' : max_depth,
        'max_features' : max_features
    }  
    
    hyper_list_2.append(h_params)

In [ ]:
pd.DataFrame.from_dict(hyper_list_2).sort_values(by='score').head(10)

In [ ]:
rf_reg_1 = RandomForestRegressor(n_jobs=-1, n_estimators=300, criterion='squared_error',
                                max_depth=8, max_features=0.585832)
rf_reg_2 = RandomForestRegressor(n_jobs=-1, n_estimators=300, criterion='squared_error',
                                max_depth=9, max_features=0.275592)

In [ ]:
rf_reg_1.fit(X_train_1, y_train_1)

In [ ]:
ftr_importance = pd.Series(rf_reg_1.feature_importances_, 
                           index = X_train_1.columns).sort_values(ascending=False)
sns.barplot(x=ftr_importance, y=ftr_importance.index)

In [ ]:
rf_reg_2.fit(X_train_2, y_train_2)

In [ ]:
pred_1 = rf_reg_1.predict(X_test_1)
pred_1

In [ ]:
pred_2 = rf_reg_2.predict(X_test_2)
pred_2

In [ ]:
submission['중식계'] = pred_1
submission.head(5)

In [ ]:
submission['석식계'] = pred_2
submission.head(5)

In [ ]:
submission.to_csv("data_dacon/dacon_submission-1.csv", index=False)

In [ ]:
tmp = ['쇠고기무국','','주꾸미볶음']

In [ ]:
tmp

In [ ]:
for n in tmp:
    print(n)

In [ ]:
len(train)

In [ ]:
train['중식메뉴'][20]

In [ ]:
menu_lunch2 = []

for i in range(0,len(train)):
    tmp = train['중식메뉴'][i].split(' ')
    for n in tmp:
        if (len(n) > 1) and ('(' not in n):
            menu_lunch2.append(n)

In [ ]:
menu_lunch2

In [ ]:
import re

In [ ]:
text1 = "(New)고구마순등뼈탕"
re.sub(r"[(New)]", "", text1)

In [ ]:
menu_lunch2

In [ ]:
menu_lunch = []

for i in range(0,len(train)):
    tmp = train['중식메뉴'][i].split(' ')
    for n in tmp:
        if len(n) > 1:
            n = re.sub(r"[(New)]", "", n)
            menu_lunch.append(n)

In [ ]:
print(len(menu_lunch), len(set(menu_lunch)))

sorted(set(menu_lunch))
pd.DataFrame(sorted(set(menu_lunch))).to_csv('data_dacon/df_lunch.csv', index=False, encoding='cp949')

In [ ]:
train.head(3)
tmp = train['중식메뉴'][3].split(' ')
for n in tmp:
    if len(n) > 1:
        print(n)

In [ ]:
tmp

In [ ]:
#!pip install lightgbm

In [ ]:
import lightgbm as lgbm

In [ ]:
lgbm_1 = lgbm.LGBMRegressor(learning_rate=0.1, n_estimators=500)
lgbm_2 = lgbm.LGBMRegressor(learning_rate=0.1, n_estimators=500)

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=777)

In [ ]:
lgbms_1 = []

for train_idx, val_idx in kfold.split(X_train_1):
    X_1 = X_train_1.iloc[train_idx]
    y_1 = y_train_1.iloc[train_idx]
    X_1_val = X_train_1.iloc[val_idx]
    y_1_val = y_train_1.iloc[val_idx]
    
    lgbms_1.append(lgbm_1.fit(X_1, y_1, eval_set=(X_1_val, y_1_val) )) #, early_stopping_rounds=100, verbose=100))

In [ ]:
lgbms_1

In [ ]:
lgbm_1.fit(X_train_1, y_train_1)

In [ ]:
pred_1 = lgbm_1.predict(X_test_1)
pred_1

In [ ]:
submission['중식계'] = pred_1
submission.head(5)

In [ ]:
lgbm_2.fit(X_train_2, y_train_2)

In [ ]:
pred_2 = lgbm_2.predict(X_test_2)
pred_2

In [ ]:
submission['석식계'] = pred_2
submission.head(5)

In [ ]:
submission.to_csv("data_dacon/dacon_submission-2.csv", index=False)